In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T
import datetime as dt
import json
import sys

dbutils.widgets.removeAll()

dbutils.widgets.text("brewdat_library_version", "v1.1.5", "01 - brewdat_library_version")
brewdat_library_version = dbutils.widgets.get("brewdat_library_version")
print(f"{brewdat_library_version = }")

dbutils.widgets.text("target_database", "gld_maz_logistics_warehouse", "02 - target_database")
target_database = dbutils.widgets.get("target_database")
print(f"{target_database = }")

dbutils.widgets.text("target_table", f"maz_stock_inventory_plus", "03 - target_table")
target_table = dbutils.widgets.get("target_table")
print(f"{target_table = }")

dbutils.widgets.text("target_zone", "maz", "04 - target_zone")
target_zone = dbutils.widgets.get("target_zone")
print(f"{target_zone = }")

dbutils.widgets.text("target_business_domain", "logistics", "05 - target_business_domain")
target_business_domain = dbutils.widgets.get("target_business_domain")
print(f"{target_business_domain = }")

dbutils.widgets.text("target_subzone", f"copecac", "06 - target_subzone")
target_subzone = dbutils.widgets.get("target_subzone")
print(f"{target_subzone = }")

dbutils.widgets.text("target_data_product", "item", "07 - target_product")
target_product = dbutils.widgets.get("target_data_product")
print(f"{target_product = }")

date_filter_end = dt.datetime.today().date()
date_filter_start = dt.datetime.today().date() - dt.timedelta(days=1)
print("date_filter_end = '{}'".format(date_filter_end))
print("date_filter_start = '{}'".format(date_filter_start))

dbutils.widgets.text("data_interval_end", "{}".format(date_filter_end), "09 - data_interval_end")
data_interval_end = dbutils.widgets.get("data_interval_end")
print("data_interval_end = '{}'".format(data_interval_end))

dbutils.widgets.text("data_interval_start", "{}".format(date_filter_start), "08 - data_interval_start")
data_interval_start = dbutils.widgets.get("data_interval_start")
print("data_interval_start = '{}'".format(data_interval_start))

dbutils.widgets.text("partition_date_format", "yyyy", "10 - partition_date_format")
partition_date_format = "yyyy"
partition_date_format = dbutils.widgets.get("partition_date_format")
print("partition_date_format = '{}'".format(partition_date_format))

dbutils.widgets.text("partition_columns", "stock_insertion_date", "11 - partition_column")
partition_column = dbutils.widgets.get("partition_columns")
print("partition_columns = '{}'".format(partition_column))

dbutils.widgets.text("mchb", "","12 - mchb")
mchb = dbutils.widgets.get("mchb")
print(f"{mchb = }")

dbutils.widgets.text("mard", "","13 - mard")
mard = dbutils.widgets.get("mard")
print(f"{mard = }")

dbutils.widgets.text("mbew", "","14 - mbew")
mbew = dbutils.widgets.get("mbew")
print(f"{mbew = }")

dbutils.widgets.text("marc", "","15 - marc")
marc = dbutils.widgets.get("marc")
print(f"{marc = }")

delta_column_name = 'target_apply_dt'
print(f"{delta_column_name = }")

In [0]:
sys.path.append(f"/Workspace/Repos/brewdat_library/{brewdat_library_version}")
from brewdat.data_engineering import common_utils, lakehouse_utils, transform_utils, write_utils
common_utils.set_global_dbutils(dbutils)

In [0]:
%run "../set_project_context"

In [0]:
import os
environment = os.getenv("ENVIRONMENT")
if environment not in ["dev", "qa", "prod"]:
    raise Exception(
        "This Databricks Workspace does not have necessary environment variables."
        " Contact the admin team to set up the global init script and restart your cluster."
    )

if environment == 'dev':
    src_uc = 'brewdat_uc_maz_dev'
elif environment == 'qa':
    src_uc = 'brewdat_uc_maz_qa'
elif environment == 'prod':
    src_uc = 'brewdat_uc_maz_prod'

if target_subzone == 'copecac':
    schema_md = 'slv_maz_masterdata_sap_pr3'
    schema_sp = 'slv_maz_supply_sap_pr3'
elif target_subzone == 'mx':
    schema_md = 'slv_maz_masterdata_sap_pr0'
    schema_sp = 'slv_maz_supply_sap_pr0'


print(f"{schema_md=}\n{schema_sp=}\n{src_uc=}")

In [0]:
src_uc='brewdat_uc_maz_prod'

In [0]:
common_utils.configure_spn_access_for_adls(
    storage_account_names=[adls_silver_gold_storage_account_name],
    key_vault_name=key_vault_name,
    spn_client_id=spn_client_id,
    spn_secret_name=spn_secret_name
)

In [0]:
spark.conf.set("spark.databricks.adaptive.autoOptimizeShuffle.minPartitionNumber", 10000)
spark.conf.set("spark.databricks.adaptive.autoOptimizeShuffle.enabled", True )
spark.conf.set("spark.databricks.adaptive.skewJoin.spillProof.enabled", True)

In [0]:
# def consulting_delta(schema_name, table_name, columns, key_columns, delta_column_name):
#     columns = ', '.join(columns)
#     key_columns=', '.join(key_columns)
    
#     delta_dataframe = spark.sql(f"""
#                             SELECT {columns}
#                             FROM {schema_name}.{table_name} 
#                             WHERE op_ind != 'D'; 
#                             """)
    
#     return delta_dataframe


def consulting_delta(table_name, columns, key_columns, delta_column_name):
    columns = ', '.join(columns)
    key_columns=', '.join(key_columns)
    
    delta_dataframe = spark.sql(f"""
                            SELECT {columns}
                            FROM {table_name}
                            WHERE op_ind != 'D'; 
                            """)
    
    return delta_dataframe

#### Creating Delta for Pivot tables

In [0]:
# print("-"+data_interval_start+"- -"+data_interval_end+"-")

# copecac_mchb = consulting_delta(schema_name = src_uc+'.'+schema_sp
#                                     , table_name = f'{target_subzone}_{mchb}'
#                                      , columns = ["matnr","werks","lgort","charg","lvorm","clabs","cspem","cinsm","cretm","ceinm","cumlm","cvmum","cvmin","cvmei","cvmsp","cvmre","cvmla","ersda","laeda","op_ind"]
#                                      , key_columns = ["matnr","werks","lgort","charg"]
#                                      , delta_column_name = delta_column_name)

print("-"+data_interval_start+"- -"+data_interval_end+"-")

copecac_mchb = consulting_delta( table_name = mchb
                                     , columns = ["matnr","werks","lgort","charg","lvorm","clabs","cspem","cinsm","cretm","ceinm","cumlm","cvmum","cvmin","cvmei","cvmsp","cvmre","cvmla","ersda","laeda","op_ind"]
                                     , key_columns = ["matnr","werks","lgort","charg"]
                                     , delta_column_name = delta_column_name)

In [0]:
copecac_mchb = copecac_mchb.filter((F.col("charg")).isNotNull()).alias("copecac_mchb")

##### Reading Catalog Tables

In [0]:

try:
    # copecac_mard = (
    #     spark.read.table(f"{src_uc}.{schema_md}.{target_subzone}_mard")
    #     .select("matnr","werks","lgort","lvorm","labst","speme","insme","retme","einme")
    #     .filter(~F.col("op_ind").contains("D"))
    #     .alias("copecac_mard")
    # ) 
    
    copecac_mard = (
        spark.read.table(mard)
        .select("matnr","werks","lgort","lvorm","labst","speme","insme","retme","einme")
        .filter(~F.col("op_ind").contains("D"))
        .alias("copecac_mard")
    ) 

    copecac_mara = (
        spark.read.table(f"{src_uc}.{schema_md}.{target_subzone}_mara")
        .select("matnr", "meins")
        .filter(~F.col("op_ind").contains("D"))
        .alias("copecac_mara")
    )
    # copecac_marc = (
    #     spark.read.table(f"{src_uc}.{schema_md}.{target_subzone}_marc")
    #     .select("matnr", "werks", "trame", "bwesb", "umlmc", "glgmg", "lvorm")
    #     .filter(~F.col("op_ind").contains("D"))
    #     .alias("copecac_marc")
    # )
    copecac_marc = (
        spark.read.table(marc)
        .select("matnr", "werks", "trame", "bwesb", "umlmc", "glgmg", "lvorm")
        .filter(~F.col("op_ind").contains("D"))
        .alias("copecac_marc")
    )
    copecac_marm = (
        spark.read.table(f"{src_uc}.{schema_md}.{target_subzone}_marm")
        .select("matnr", "meinh", "umren", "umrez")
        .filter(~F.col("op_ind").contains("D"))
        .alias("copecac_marm")
    )
    # copecac_mbew = (
    #     spark.read.table(f"{src_uc}.{schema_md}.{target_subzone}_mbew")
    #     .select("matnr", "bwkey", "stprs", "vmpei")
    #     .filter(~F.col("op_ind").contains("D"))
    #     .alias("copecac_mbew")
    # )
    copecac_mbew = (
        spark.read.table(mbew)
        .select("matnr", "bwkey", "salk3", "lbkum", "vprsv", "verpr", "stprs", "vmpei", "peinh", "bwtar")
        .filter(~F.col("op_ind").contains("D"))
        .alias("copecac_mbew")
    )

    # TODO. Así estaba... 
    # copecac_mbew = transform_utils.deduplicate_records(df=copecac_mbew, key_columns=["matnr", "bwkey", "stprs", "vmpei"])

    # TODO. "Así es" La verdad es que se debería usar una tabla intermedia para poder saber que bwtar es el
    # que maneja cada lote, pero eso no esta planteado en esta logica (Por ende tomo el valor sin lote..)
    copecac_mbew = copecac_mbew.filter(F.trim(F.col("bwtar")) == "").drop("bwtar")
    
    copecac_mbew = (
        copecac_mbew
        .withColumn(
            "inv_unitary_price",
            F.when(
                (F.col("salk3").isNotNull()) & (F.col("lbkum") != 0),
                F.col("salk3") / F.col("lbkum")
            )
            .when(
                (F.col("vprsv") == "S") & F.col("stprs").isNotNull() & (F.col("peinh") != 0),
                F.col("stprs") / F.col("peinh")
            )
            .when(
                (F.col("vprsv") == "V") & F.col("verpr").isNotNull() & (F.col("peinh") != 0),
                F.col("verpr") / F.col("peinh")
            )
            .otherwise(F.lit(0))
        )
        .drop("salk3", "lbkum", "vprsv", "verpr", "peinh")
    )
    
    # ---

    copecac_t001w = (
        spark.read.table(f"{src_uc}.{schema_md}.{target_subzone}_t001w")
        .select("werks", "vkorg", "name1","land1","op_ind")
        .filter((F.col("op_ind") != "D") & (F.trim(F.col("spras")) == 'S'))
        .drop("op_ind")
        .alias("copecac_t001w")
    )

    copecac_mchb = copecac_mchb.join(
            copecac_t001w,
            (F.trim(F.col("copecac_mchb.werks")) == F.trim(F.col("copecac_t001w.werks"))),
            "left",)\
            .drop(copecac_t001w['werks'],copecac_t001w['name1'],copecac_t001w['vkorg']).alias("copecac_mchb")

    copecac_mard = copecac_mard.join(
            copecac_t001w,
            (F.trim(F.col("copecac_mard.werks")) == F.trim(F.col("copecac_t001w.werks"))),
            "left",)\
            .drop(copecac_t001w['werks'],copecac_t001w['name1'],copecac_t001w['vkorg']).alias("copecac_mard")

    copecac_marc = copecac_marc.join(
            copecac_t001w,
            (F.trim(F.col("copecac_marc.werks")) == F.trim(F.col("copecac_t001w.werks"))),
            "left",)\
            .drop(copecac_t001w['werks'],copecac_t001w['name1'],copecac_t001w['vkorg']).alias("copecac_marc")

except Exception as e:
    print(e)
    common_utils.exit_with_last_exception()

In [0]:
try:
    sub_query = (
        copecac_mard
        .join(
            copecac_mchb,
            (
                (F.trim(F.col("copecac_mard.matnr")) == F.trim(F.col("copecac_mchb.matnr")))
                & (F.trim(F.col("copecac_mard.werks")) == F.trim(F.col("copecac_mchb.werks")))
                & (F.trim(F.col("copecac_mard.lgort")) == F.trim(F.col("copecac_mchb.lgort")))
            ),
            "left",
        )
        .select(
            (F.col("copecac_mchb.land1")).alias("land1_mchb"),
            (F.col("copecac_mchb.matnr")).alias("matnr_mchb"),
            (F.col("copecac_mchb.werks")).alias("werks_mchb"),
            (F.col("copecac_mchb.lgort")).alias("lgort_mchb"),
            (F.col("copecac_mchb.charg")).alias("charg_mchb"),
            (F.col("copecac_mchb.lvorm")).alias("lvorm_mchb"),
            (F.col("copecac_mchb.clabs")).alias("clabs_mchb"),
            (F.col("copecac_mchb.cspem")).alias("cspem_mchb"),
            (F.col("copecac_mchb.cinsm")).alias("cinsm_mchb"),
            (F.col("copecac_mchb.cretm")).alias("cretm_mchb"),
            (F.col("copecac_mchb.ceinm")).alias("ceinm_mchb"),
            (F.col("copecac_mchb.cumlm")).alias("cumlm_mchb"),
            (F.col("copecac_mchb.cvmum")).alias("cvmum_mchb"),
            (F.col("copecac_mchb.cvmin")).alias("cvmin_mchb"),
            (F.col("copecac_mchb.cvmei")).alias("cvmei_mchb"),
            (F.col("copecac_mchb.cvmsp")).alias("cvmsp_mchb"),
            (F.col("copecac_mchb.cvmre")).alias("cvmre_mchb"),
            (F.col("copecac_mchb.cvmla")).alias("cvmla_mchb"),
            (F.col("copecac_mchb.ersda")).alias("ersda_mchb"),
            (F.col("copecac_mchb.laeda")).alias("laeda_mchb"),
            (F.col("copecac_mchb.op_ind")).alias("op_ind_mchb"),
            (F.col("copecac_mard.land1")).alias("land1_mard"),
            (F.col("copecac_mard.matnr")).alias("matnr_mard"),
            (F.col("copecac_mard.werks")).alias("werks_mard"),
            (F.col("copecac_mard.lgort")).alias("lgort_mard"),
            (F.col("copecac_mard.lvorm")).alias("lvorm_mard"),
            (F.col("copecac_mard.labst")).alias("labst_mard"),
            (F.col("copecac_mard.speme")).alias("speme_mard"),
            (F.col("copecac_mard.insme")).alias("insme_mard"),
            (F.col("copecac_mard.retme")).alias("retme_mard"),
            (F.col("copecac_mard.einme")).alias("einme_mard")
        )
        .alias("sub_query")
    )

except Exception:
    common_utils.exit_with_last_exception()

In [0]:
sub_query_0 = sub_query.select(
            (F.col("matnr_mard")).alias("matnr"),
            (F.col("werks_mard")).alias("werks"),
            (F.col("lgort_mard")).alias("lgort"),
            (F.col("land1_mard")).alias("land1"),
            (F.col("charg_mchb")).alias("charg"),
            F.coalesce(F.col("lvorm_mchb"), F.col("lvorm_mard")).alias("lvorm"),
            (F.col("lvorm_mchb")).alias("lvorm"),
            (F.col("clabs_mchb")).alias("clabs"),
            (F.col("cspem_mchb")).alias("cspem"),
            (F.col("cinsm_mchb")).alias("cinsm"),
            (F.col("cretm_mchb")).alias("cretm"),
            (F.col("ceinm_mchb")).alias("ceinm"),
            (F.col("cumlm_mchb")).alias("cumlm"),
            (F.col("cvmum_mchb")).alias("cvmum"),
            (F.col("cvmin_mchb")).alias("cvmin"),
            (F.col("cvmei_mchb")).alias("cvmei"),
            (F.col("cvmsp_mchb")).alias("cvmsp"),
            (F.col("cvmre_mchb")).alias("cvmre"),
            (F.col("cvmla_mchb")).alias("cvmla"),
            (F.col("ersda_mchb")).alias("ersda"),
            (F.col("laeda_mchb")).alias("laeda"),
            (F.col("labst_mard")).alias("labst"),
            (F.col("speme_mard")).alias("speme"),
            (F.col("insme_mard")).alias("insme"),
            (F.col("retme_mard")).alias("retme"),
            (F.col("einme_mard")).alias("einme"),
            (F.col("op_ind_mchb")).alias("op_ind")
        ).alias("sub_query_0")

In [0]:
sub_query_0 = transform_utils.deduplicate_records(df=sub_query_0, key_columns=["matnr", "werks", "lgort", "charg"], watermark_column = ['ersda','laeda'])

In [0]:
try:
    sub_query_1 = (
        sub_query_0
        .join(
            copecac_mara,
            (
                F.trim(F.col("sub_query_0.matnr")) == F.trim(F.col("copecac_mara.matnr"))
            ),
            "left",
        )
        .select(
            "sub_query_0.*", 
            F.trim(F.col("copecac_mara.meins")).alias("meins")
        )
        .alias("sub_query_1")
    )

except Exception:
    common_utils.exit_with_last_exception()

In [0]:
try:
    sub_query_2 = (
        sub_query_1
        .join(
            copecac_marc,
            (
                (F.trim(F.col("sub_query_1.matnr")) == F.trim(F.col("copecac_marc.matnr")))
                & (F.trim(F.col("sub_query_1.werks")) == F.trim(F.col("copecac_marc.werks")))
            ),
            "left",
        )
        .select(
            "sub_query_1.*",
            F.trim(F.col("copecac_marc.trame")).alias("trame"),
            F.trim(F.col("copecac_marc.bwesb")).alias("bwesb"),
            F.trim(F.col("copecac_marc.umlmc")).alias("umlmc"),
            F.trim(F.col("copecac_marc.glgmg")).alias("glgmg")
        )
        .alias("sub_query_2")
    )
    sub_query_1.unpersist()

except Exception:
    common_utils.exit_with_last_exception()

In [0]:
try:
    sub_query_3 = (
        sub_query_2
        .join(
            copecac_mbew,
            (
                (F.trim(F.col("sub_query_2.matnr")) == F.trim(F.col("copecac_mbew.matnr")))
                & (F.trim(F.col("sub_query_2.werks")) == F.trim(F.col("copecac_mbew.bwkey")))
            ),
            "left",
        )
        .select(
            "sub_query_2.*",
            F.trim(F.col("copecac_mbew.stprs")).alias("stprs"),
            F.trim(F.col("copecac_mbew.vmpei")).alias("vmpei"),
            copecac_mbew["inv_unitary_price"].alias("inv_unitary_price")
        )
        .alias("sub_query_3")
    )
    sub_query_2.unpersist()

except Exception as e:
    print(e)
    common_utils.exit_with_last_exception()

In [0]:
try:
    new_cols2 = {
                "country_code": (F.when((F.lit(target_subzone) == 'mx'), F.lit('MX')).otherwise(F.col('sub_query_3.land1'))).cast(T.StringType()),
                "product_code": F.trim(F.col('sub_query_3.matnr')).cast(T.StringType()),
                "product_short_code": F.regexp_replace(F.col('sub_query_3.matnr'), r'^[0]*', '').cast(T.StringType()),
                "plant_code": F.trim(F.col('sub_query_3.werks')).cast(T.StringType()),
                "storage_location_code": F.trim(F.col('sub_query_3.lgort')).cast(T.StringType()),
                "batch_code": F.col('sub_query_3.charg').cast(T.StringType()),
                "unitary_price": ((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType()))).cast(T.DecimalType(13,3)),
                "inv_unitary_price": F.col('sub_query_3.inv_unitary_price').cast(T.DecimalType(13,3)),

                "unrestricted": (F.when((F.col('sub_query_3.charg').isNull()), F.lit(F.col('sub_query_3.labst'))).otherwise(F.lit(F.col('sub_query_3.clabs')))).cast(T.DecimalType(13,3)),
                "unrestricted_hl": (((F.when((F.col('sub_query_3.charg').isNull()), F.lit(F.col('sub_query_3.labst'))).otherwise(F.lit(F.col('sub_query_3.clabs')))))* ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "unrestricted_ca": (((F.when((F.col('sub_query_3.charg').isNull()), F.lit(F.col('sub_query_3.labst'))).otherwise(F.lit(F.col('sub_query_3.clabs')))))* ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "unrestricted_pal": (((F.when((F.col('sub_query_3.charg').isNull()), F.lit(F.col('sub_query_3.labst'))).otherwise(F.lit(F.col('sub_query_3.clabs')))))* ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_unrestricted": ((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType())) *  (F.when((F.col('sub_query_3.charg').isNull()), F.lit(F.col('sub_query_3.labst'))).otherwise(F.lit(F.col('sub_query_3.clabs'))))).cast(T.DecimalType(13,3)),

                "blocked": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.speme'))).otherwise(F.lit(F.col('sub_query_3.cspem')))).cast(T.DecimalType(13,3)),
                "blocked_hl": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.speme'))).otherwise(F.lit(F.col('sub_query_3.cspem')))))* ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "blocked_ca": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.speme'))).otherwise(F.lit(F.col('sub_query_3.cspem')))))* ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "blocked_pal": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.speme'))).otherwise(F.lit(F.col('sub_query_3.cspem')))))* ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_blocked_stock":  ((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType())) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.speme'))).otherwise(F.lit(F.col('sub_query_3.cspem'))))).cast(T.DecimalType(13,3)),

                "in_quality_inspection": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.insme'))).otherwise(F.lit(F.col('sub_query_3.cinsm')))).cast(T.DecimalType(13,3)), 
                "in_quality_inspection_hl": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.insme'))).otherwise(F.lit(F.col('sub_query_3.cinsm')))))* ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "in_quality_inspection_ca": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.insme'))).otherwise(F.lit(F.col('sub_query_3.cinsm')))))* ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "in_quality_inspection_pal": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.insme'))).otherwise(F.lit(F.col('sub_query_3.cinsm')))))* ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_in_quality_inspection": ((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType())) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.insme'))).otherwise(F.lit(F.col('sub_query_3.cinsm'))))).cast(T.DecimalType(13,3)),

                "transit_transf": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum')))).cast(T.DecimalType(13,3)),
                "transit_transf_hl": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum')))))* ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "transit_transf_ca": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum')))))* ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "transit_transf_pal": (((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum')))))* ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "val_in_trans":      ((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType())) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame'))).otherwise(F.lit(F.col('sub_query_3.cvmum'))))).cast(T.DecimalType(13,3)),

                "returns": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.retme'))).otherwise(F.lit(F.col('sub_query_3.cretm')))).cast(T.DecimalType(13,3)), 
                "returns_hl": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.retme'))).otherwise(F.lit(F.col('sub_query_3.cretm'))) * ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "returns_ca": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.retme'))).otherwise(F.lit(F.col('sub_query_3.cretm'))) * ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "returns_pal": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.retme'))).otherwise(F.lit(F.col('sub_query_3.cretm'))) * ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_rets_blocked":  ((F.col('sub_query_3.stprs').cast(T.FloatType()) / F.col('sub_query_3.vmpei').cast(T.FloatType())) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.retme'))).otherwise(F.lit(F.col('sub_query_3.cretm'))))).cast(T.DecimalType(13,3)),

                "restricted_use": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.einme'))).otherwise(F.lit(F.col('sub_query_3.ceinm')))).cast(T.DecimalType(13,3)),
                "restricted_use_hl": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.einme'))).otherwise(F.lit(F.col('sub_query_3.ceinm'))) * ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "restricted_use_ca": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.einme'))).otherwise(F.lit(F.col('sub_query_3.ceinm'))) * ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "restricted_use_pal": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.einme'))).otherwise(F.lit(F.col('sub_query_3.ceinm'))) * ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_restricted": (((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType()))) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.einme'))).otherwise(F.lit(F.col('sub_query_3.ceinm'))))).cast(T.DecimalType(13,3)),
                "val_grbs": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.bwesb')))    .otherwise(F.lit(F.col('sub_query_3.glgmg')))).cast(T.DecimalType(13,3)),
                "val_grbs_hl": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.bwesb')))    .otherwise(F.lit(F.col('sub_query_3.glgmg'))) * ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "val_grbs_ca": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.bwesb')))    .otherwise(F.lit(F.col('sub_query_3.glgmg'))) * ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "val_grbs_pal": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.bwesb')))    .otherwise(F.lit(F.col('sub_query_3.glgmg'))) * ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "total_value":         (((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType()))) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.bwesb'))).otherwise(F.lit(F.col('sub_query_3.glgmg'))))).cast(T.DecimalType(13,3)),
                "stock_in_transit":          (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum')))).cast(T.DecimalType(13,3)),
                "stock_in_transit_hl": ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum'))))* ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "stock_in_transit_ca": ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum'))))* ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "stock_in_transit_pal": ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame')))    .otherwise(F.lit(F.col('sub_query_3.cvmum'))))* ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_in_transit":      (((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType()))) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.trame'))).otherwise(F.lit(F.col('sub_query_3.cvmum'))))).cast(T.DecimalType(13,3)),
                "stock_in_trans":       (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.umlmc')))    .otherwise(F.lit(F.col('sub_query_3.cumlm')))).cast(T.DecimalType(13,3)),
                "stock_in_trans_hl": ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.umlmc')))    .otherwise(F.lit(F.col('sub_query_3.cumlm'))))* ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "stock_in_trans_ca": ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.umlmc')))    .otherwise(F.lit(F.col('sub_query_3.cumlm'))))* ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                 "stock_in_trans_pal": (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.umlmc')))    .otherwise(F.lit(F.col('sub_query_3.cumlm')))* ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_in_stock_tfr":      (((F.col('sub_query_3.stprs').cast(T.FloatType())) / (F.col('sub_query_3.vmpei').cast(T.FloatType()))) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.umlmc'))).otherwise(F.lit(F.col('sub_query_3.cumlm'))))).cast(T.DecimalType(13,3)), 
                "tied_empties":     (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.glgmg')))    .otherwise(F.lit(F.col('sub_query_3.cumlm')))).cast(T.DecimalType(13,3)),
                "tied_empties_hl":  ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.glgmg')))    .otherwise(F.lit(F.col('sub_query_3.cumlm'))))* ((F.trim(F.col('marm_destino_HL.umren')) / F.trim(F.col('marm_destino_HL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                 "tied_empties_ca": ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.glgmg')))    .otherwise(F.lit(F.col('sub_query_3.cumlm'))))* ((F.trim(F.col('marm_destino_CA.umren')) / F.trim(F.col('marm_destino_CA.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "tied_empties_pal": ((F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.glgmg')))    .otherwise(F.lit(F.col('sub_query_3.cumlm'))))* ((F.trim(F.col('marm_destino_PAL.umren')) / F.trim(F.col('marm_destino_PAL.umrez'))) / (F.trim(F.col('marm_origen.umren')) / F.trim(F.col('marm_origen.umrez'))))).cast(T.DecimalType(13,3)),
                "value_tied_empties":   ((F.col('sub_query_3.stprs').cast(T.FloatType()) / (F.col('sub_query_3.vmpei').cast(T.FloatType()))) * (F.when(F.col("sub_query_3.charg").isNull(), F.lit(F.col('sub_query_3.glgmg'))).otherwise(F.lit(F.col('sub_query_3.cumlm'))))).cast(T.DecimalType(13,3)),
                
                "stock_insertion_date": (F.when((F.col('sub_query_3.ersda').isNull()), F.lit(F.current_date())).otherwise(F.lit(F.col('sub_query_3.ersda')))).cast(T.DateType()),
                "stock_update_date": (F.when((F.col('sub_query_3.laeda').isNull()), F.lit('1900-01-01')).otherwise(F.lit(F.col('sub_query_3.laeda')))).cast(T.DateType()),
                "key": 
                    F.trim(F.concat(
                        (F.when((F.trim(F.col('sub_query_3.matnr')).isNull()), "Null").otherwise((F.trim(F.col('sub_query_3.matnr'))).cast(T.StringType()))),
                        (F.when((F.trim(F.col('sub_query_3.werks')).isNull()), "Null").otherwise((F.trim(F.col('sub_query_3.werks'))).cast(T.StringType()))),
                        (F.when((F.trim(F.col('sub_query_3.lgort')).isNull()), "Null").otherwise((F.trim(F.col('sub_query_3.lgort'))).cast(T.StringType()))),
                        (F.when(((F.col('sub_query_3.charg')).isNull()), "Null").otherwise(((F.col('sub_query_3.charg'))).cast(T.StringType())))
                    )),
                "__partition_column": F.date_format(F.when((F.col('sub_query_3.ersda').isNull()), F.lit(F.current_date())).otherwise(F.lit(F.col('sub_query_3.ersda'))).cast('date'), partition_date_format).cast(T.IntegerType()),
                "op_ind": F.col('op_ind')
                }

except Exception as e:
    print(e)
    common_utils.exit_with_last_exception()

In [0]:
try:
    sub_query_4 = (
        sub_query_3
        .join(
            F.broadcast(copecac_marm).alias("marm_origen"),
            (
                (
                    F.trim(F.col("sub_query_3.matnr")) == F.trim(F.col("marm_origen.matnr"))
                ).alias("condition1")
                & (
                    F.trim(F.col("marm_origen.meinh")) == F.trim(F.col("sub_query_3.meins"))
                ).alias("condition2")
            ),
            "left",
        )
        .join(
            F.broadcast(copecac_marm).alias("marm_destino_HL"),
            (
                (
                    F.trim(F.col("sub_query_3.matnr")) == F.trim(F.col("marm_destino_HL.matnr"))
                ).alias("condition3")
                & 
                (
                    F.trim(F.upper(F.col("marm_destino_HL.meinh"))) == "HL"
                ).alias("condition4")
            ),
            "left",
        )
        .join(
            F.broadcast(copecac_marm).alias("marm_destino_CA"),
            (
                (
                    F.trim(F.col("sub_query_3.matnr")) == F.trim(F.col("marm_destino_CA.matnr"))
                ).alias("condition5")
                & (
                    F.trim(F.upper(F.col("marm_destino_CA.meinh"))) == "CA"
                ).alias( "condition6")
            ),
            "left",
        )
        .join(
            F.broadcast(copecac_marm).alias("marm_destino_PAL"),
            (
                (
                    F.trim(F.col("sub_query_3.matnr")) == F.trim(F.col("marm_destino_PAL.matnr"))
                ).alias("condition7")
                & 
                (
                    F.trim(F.upper(F.col("marm_destino_PAL.meinh"))) == "PAL"
                ).alias( "condition8")
            ),
            "left",
        )
        .withColumns(new_cols2)
        .select(list(new_cols2.keys()))
        .alias("sub_query_4")
    )

except Exception as e:
    print(e)
    common_utils.exit_with_last_exception()

In [0]:
final_df = sub_query_4.na.fill(0, subset=["unitary_price","inv_unitary_price","unrestricted_hl","unrestricted_ca","unrestricted_pal", "value_unrestricted","blocked_hl","blocked_ca","blocked_pal","value_blocked_stock","in_quality_inspection_hl","in_quality_inspection_ca","in_quality_inspection_pal","value_in_quality_inspection","transit_transf_hl","transit_transf_ca","transit_transf_pal","val_in_trans","returns_hl","returns_ca","returns_pal","value_rets_blocked","restricted_use_hl","restricted_use_ca","restricted_use_pal","value_restricted","val_grbs_hl","val_grbs_ca","val_grbs_pal","total_value","stock_in_transit_hl","stock_in_transit_ca","stock_in_transit_pal","value_in_transit","stock_in_trans_hl","stock_in_trans_ca","stock_in_trans_pal","value_in_stock_tfr","tied_empties_hl","tied_empties_ca","tied_empties_pal","value_tied_empties"])
final_df = final_df.na.fill('NA', subset=["country_code","product_code","product_short_code","plant_code","storage_location_code","batch_code"])
final_df = final_df.filter(~((F.col("country_code") == 'BO') | (F.col("country_code") == 'NA')))

In [0]:
try:
    audit_df = transform_utils.create_or_replace_audit_columns(final_df)
    audit_df = audit_df.drop(F.col("op_ind"))
    
except Exception:
    common_utils.exit_with_last_exception()

In [0]:
# Sets location for gold folder --revisar
params_list = [lakehouse_gold_root, target_zone, target_business_domain,
                   target_subzone,target_product, target_table]

if any(x is None or len(x) == 0 for x in params_list):
        raise ValueError("Location would contain null or empty values.")

lakehouse_utils.assert_valid_zone(target_zone)
lakehouse_utils.assert_valid_business_domain(target_business_domain)
lakehouse_utils.assert_valid_folder_name(target_table)
target_location = (f"{lakehouse_gold_root}/data/{target_zone}/{target_business_domain}/" +
            f"gld_{target_zone}_{target_business_domain}_warehouse/{target_table}").lower()

print(f"{target_location = }")
print(f"{target_table = }")
print(f"{lakehouse_gold_root = }")

In [0]:
auditZeros = audit_df.filter((F.col("unrestricted") + F.col("blocked") + F.col("in_quality_inspection") + F.col("returns") + F.col("restricted_use")) <= 0)
audit_df = audit_df.subtract(auditZeros)

In [0]:
if target_subzone == 'mx':
    load_type = 'OVERWRITE_TABLE'
if target_subzone == 'copecac':
    load_type = 'APPEND_ALL'

In [0]:
results = write_utils.write_delta_table(
    df=audit_df,
    location=target_location,
    database_name=target_database,
    table_name=target_table,
    # load_type=load_type,
    load_type='OVERWRITE_TABLE',
    key_columns=["country_code","product_code","product_short_code","plant_code","storage_location_code","batch_code"],
    partition_columns=["__partition_column"],
    schema_evolution_mode=write_utils.SchemaEvolutionMode.ADD_NEW_COLUMNS
)
print(results)

In [0]:
common_utils.exit_with_object(results)